In [1]:
#-*-coding:utf-8-*-
import pandas as pd
import math
import scipy.stats as st
from scipy.special import gamma
import numpy as np

from datetime import datetime
import xlrd

## 一、BNS方法识别指数跳跃
1. $\Delta$：日内高频价格采样时长，论文中选取10min
2. $M$：相应的日内采样间隔数
3. $p_{t,j}$：$t$日第$j$个长度为$\Delta$的时间间隔末的对数价格
4. $r_{t,j}$：对数收益率，$r_{t,j}=p_{t,j}-p_{t, j-1}$
5. $RV_t$：$t$日已实现波动率，$RV_t=\sum_{j=1}^M r_{t,j}^2$
6. $BPV_t$：$t$日已实现双幂次变差，$BPV_t=\frac{\pi}{2}(\frac{M}{M-1})\sum_{j=2}^M|r_{t.j}||r_{t,j-1}|$

step1: 计算Z统计量
$$Z_t=\frac{(RV_t-BPV_t)/RV_t}{\sqrt{((\frac{\pi}{2})^2+\pi-5)\frac{1}{M}max(1,\frac{TPQ_t}{BPV_t^2})}}$$
step2: 选取置信度$\alpha$，得$t$日显著离散跳跃的估计值$J_t$和连续波动的估计值$C_t$
$$J_t=I(Z_t>\phi_\alpha)(RV_t-BPV_t)$$
$$C_t=I(Z_t\leq \phi_{\alpha})RV_t+I(Z_t>\phi_{\alpha})BPV_t$$

In [2]:
## 读取指数数据(2011-01-04 ~ 2021-11-4)
ind_all = pd.read_csv('ind_all.csv', encoding='gbk')
ind_all = ind_all[['StockID', 'date', 'close']]
ind_all.columns = ['code', 'datetime', 'price']

# 时间格式转换
datetime = ind_all['datetime'].to_list()
date = []
time = []
for i in range(len(datetime)):
    dt = float(datetime[i])
    dt = xlrd.xldate.xldate_as_datetime(dt, 0)
    d = dt.__format__('%Y-%m-%d')
    t = dt.__format__('%X')
    date.append(d)
    time.append(t)
ind_all['date'] = date
ind_all['time'] = time
ind_all.drop('datetime', axis=1)
ind_all = ind_all[['code', 'date', 'time', 'price']]

# 取2011-01-04 ~ 2013-07-11之间的数据
ind = ind_all[ind_all['date'] < '2013-07-12']
ind

,code,date,time,price
0,SH000300,2011-01-04,09:35:00,3147.446
1,SH000300,2011-01-04,09:40:00,3146.459
2,SH000300,2011-01-04,09:45:00,3154.728
3,SH000300,2011-01-04,09:50:00,3153.279
4,SH000300,2011-01-04,09:55:00,3154.060
...,...,...,...,...
29227,SH000300,2013-07-11,14:40:00,2326.290
29228,SH000300,2013-07-11,14:45:00,2327.559
29229,SH000300,2013-07-11,14:50:00,2323.287
29230,SH000300,2013-07-11,14:55:00,2324.604


In [3]:
## 5分钟数据提取10分钟数据

# 10分钟采样时点
time = ind['time'].unique().tolist()
time_10 = []
for i in range(1, len(time), 2):
    time_10.append(time[i])

# 筛选
ind_10 = ind[ind['time'] == time_10[0]]
for i in range(1, len(time_10)):
    df = ind[ind['time'] == time_10[i]]
    ind_10 = pd.concat([ind_10, df])
ind_10 = ind_10.sort_values(by=['date', 'time'], ascending=[True, True], ignore_index=True)
ind_10

,code,date,time,price
0,SH000300,2011-01-04,09:40:00,3146.459
1,SH000300,2011-01-04,09:50:00,3153.279
2,SH000300,2011-01-04,10:00:00,3144.274
3,SH000300,2011-01-04,10:10:00,3145.429
4,SH000300,2011-01-04,10:20:00,3167.451
...,...,...,...,...
14611,SH000300,2013-07-11,14:20:00,2333.006
14612,SH000300,2013-07-11,14:30:00,2333.006
14613,SH000300,2013-07-11,14:40:00,2326.290
14614,SH000300,2013-07-11,14:50:00,2323.287


In [4]:
# save
ind = ind_10

In [5]:
## 计算对数价格
ind['p'] = ind.apply(lambda x: math.log(x['price']), axis=1)

## 计算对数收益率
ind['r'] = ind.groupby('date')['p'].diff()

ind['r2'] = ind.apply(lambda x: x['r']**2, axis=1)
ind

,code,date,time,price,p,r,r2
0,SH000300,2011-01-04,09:40:00,3146.459,8.054033,NaN,NaN
1,SH000300,2011-01-04,09:50:00,3153.279,8.056198,0.002165,4.687962e-06
2,SH000300,2011-01-04,10:00:00,3144.274,8.053338,-0.002860,8.178701e-06
3,SH000300,2011-01-04,10:10:00,3145.429,8.053706,0.000367,1.348850e-07
4,SH000300,2011-01-04,10:20:00,3167.451,8.060682,0.006977,4.867679e-05
...,...,...,...,...,...,...,...
14611,SH000300,2013-07-11,14:20:00,2333.006,7.754913,-0.005315,2.824497e-05
14612,SH000300,2013-07-11,14:30:00,2333.006,7.754913,0.000000,0.000000e+00
14613,SH000300,2013-07-11,14:40:00,2326.290,7.752030,-0.002883,8.310772e-06
14614,SH000300,2013-07-11,14:50:00,2323.287,7.750738,-0.001292,1.668568e-06


In [6]:
ind = ind[~ind.isna().any(axis=1)]
ind

,code,date,time,price,p,r,r2
1,SH000300,2011-01-04,09:50:00,3153.279,8.056198,0.002165,4.687962e-06
2,SH000300,2011-01-04,10:00:00,3144.274,8.053338,-0.002860,8.178701e-06
3,SH000300,2011-01-04,10:10:00,3145.429,8.053706,0.000367,1.348850e-07
4,SH000300,2011-01-04,10:20:00,3167.451,8.060682,0.006977,4.867679e-05
5,SH000300,2011-01-04,10:30:00,3172.073,8.062141,0.001458,2.126213e-06
...,...,...,...,...,...,...,...
14611,SH000300,2013-07-11,14:20:00,2333.006,7.754913,-0.005315,2.824497e-05
14612,SH000300,2013-07-11,14:30:00,2333.006,7.754913,0.000000,0.000000e+00
14613,SH000300,2013-07-11,14:40:00,2326.290,7.752030,-0.002883,8.310772e-06
14614,SH000300,2013-07-11,14:50:00,2323.287,7.750738,-0.001292,1.668568e-06


In [7]:
## 计算已实现波动率
ind_day = ind.groupby('date', as_index=False)['r2'].sum()
ind_day.columns = ['date', 'RV']


## 计算BPV
def cal_BPV(df):
    
    r = df['r'].to_list()
    M = len(r)
    
    count = 0
    for i in range(1, M):
        count = count + abs(r[i]) * abs(r[i-1])
    BPV = math.pi / 2 * (M / (M-1)) * count
    
    return BPV

ind_BPV = ind.groupby('date', as_index=False).apply(cal_BPV)
ind_BPV.columns = ['date', 'BPV']
ind_day = pd.merge(ind_day, ind_BPV, on='date', how='left')


## 计算TPQ
def cal_TPQ(df):
    
    r = df['r'].to_list()
    M = len(r)
    
    count = 0
    for i in range(M-2):
        count = count + abs(r[i])**(4/3) *  abs(r[i+1])**(4/3) *  abs(r[i+2])**(4/3)
    const = 0.25 * (gamma(7/6) / gamma(1/2))**(-3)
    TPQ = const * M**2 / (M - 2) * count
    
    return TPQ

ind_TPQ = ind.groupby('date', as_index=False).apply(cal_TPQ)
ind_TPQ.columns = ['date', 'TPQ']
ind_day = pd.merge(ind_day, ind_TPQ, on='date', how='left')


## 计算Z统计量

M = 23

ind_day['Z'] = ind_day.apply(lambda x: ((x['RV'] - x['BPV']) / x['RV']) / math.sqrt(((math.pi / 2)**2 + math.pi - 5) / M * max(1, x['TPQ']/x['BPV']**2)), axis=1)


## 显著离散跳跃估计值

alpha = 0.99 # 选择置信水平
phi = st.norm.ppf(alpha)
date = ind_day['date'].to_list()

Z = ind_day['Z'].to_list()
RV = ind_day['RV'].to_list()
BPV = ind_day['BPV'].to_list()

J = []
for i in range(len(date)):
    if Z[i] > phi:
        J.append(RV[i] - BPV[i])
    else:
        J.append(0)
ind_day['J'] = J


## 连续波动的估计值
C = []
for i in range(len(date)):
    if Z[i] <= phi:
        C.append(RV[i])
    else:
        C.append(BPV[i])
ind_day['C'] = C
ind_day

,date,RV,BPV,TPQ,Z,J,C
0,2011-01-04,0.000126,0.000102,5.829525e-09,1.149615,0.0,0.000126
1,2011-01-05,0.000095,0.000078,6.171608e-09,1.081164,0.0,0.000095
2,2011-01-06,0.000102,0.000092,9.430170e-09,0.554526,0.0,0.000102
3,2011-01-07,0.000259,0.000224,2.788767e-08,0.828077,0.0,0.000259
4,2011-01-10,0.000123,0.000144,1.754208e-08,-1.062877,0.0,0.000123
...,...,...,...,...,...,...,...
604,2013-07-05,0.000107,0.000114,1.291494e-08,-0.373776,0.0,0.000107
605,2013-07-08,0.000299,0.000309,5.498748e-08,-0.211485,0.0,0.000299
606,2013-07-09,0.000117,0.000140,3.199722e-08,-0.981177,0.0,0.000117
607,2013-07-10,0.000239,0.000259,8.232915e-08,-0.459043,0.0,0.000239


In [8]:
st.norm.ppf(0.99)

2.3263478740408408

In [9]:
jump_day = ind_day[ind_day['J'] > 0]['date']
print('（10min高频数据）指数跳跃的交易日天数为：%d' % len(jump_day))

（10min高频数据）指数跳跃的交易日天数为：22


In [10]:
jump_day.to_csv('jump_day.csv')

In [12]:
# 估计的J输出
J_BNS = ind_day[['date', 'J']]
J_BNS.to_csv('J_BNS.csv', index=False)